In [ ]:
# # !pip install fastembed langchain langchain_community PyMuPDF chromadb

# from langchain_community.document_loaders import PyMuPDFLoader
# from pathlib import Path

# data_folder = Path("../data/pdf")

# pdf_paths = list(data_folder.glob("*.pdf"))
# data_pdf = []

# for path in pdf_paths:
#     loader = PyMuPDFLoader(path)
#     data_pdf.extend(loader.load())  # Une todos los documentos

# print(f"Se cargaron {len(data_pdf)} páginas de PDFs.")


Se cargaron 912 páginas de PDFs.


In [4]:
from langchain_openai import AzureChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma


endpoint = "https://aleja-mavdk5nu-eastus2.cognitiveservices.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"
subscription_key = 'D55vuDUYoqo7Bzj9R0SwUhQFA8MZ9l2Eh5yWCOfKSlMXLmKw7JBpJQQJ99BEACHYHv6XJ3w3AAAAACOGRJuO'
api_version = "2024-12-01-preview"


endpoint_embedding = "https://aleja-mavdk5nu-eastus2.cognitiveservices.azure.com/"
deployment_embedding = "text-embedding-3-small"
api_version_embedding = "2024-02-01"

llm = AzureChatOpenAI(
    openai_api_key=subscription_key,
    azure_endpoint=endpoint,
    api_version=api_version,
    azure_deployment=deployment,
    temperature=0
)

# Configurar embeddings para Azure
embedding = AzureOpenAIEmbeddings(
    azure_deployment=deployment_embedding,
    api_key=subscription_key,
    azure_endpoint=endpoint_embedding,
    openai_api_version=api_version_embedding
)

In [2]:
from langchain.schema import HumanMessage

def generate_from_azure_openai(
    prompt: str,
    deployment: str,
    endpoint: str,
    api_key: str,
    api_version: str,
    temperature: float = 0.0
) -> str:
    """
    Se conecta a Azure OpenAI y genera una respuesta a un prompt.

    Args:
        prompt (str): El texto de entrada para el modelo.
        deployment (str): El nombre del despliegue del modelo en Azure.
        endpoint (str): El endpoint del servicio de Azure Cognitive Services.
        api_key (str): La clave de API para autenticarse.
        api_version (str): La versión de la API de Azure OpenAI a utilizar.
        temperature (float): La temperatura para la generación (controla la creatividad).

    Returns:
        str: Un string con la respuesta generada o un mensaje de error.
    """
    try:
        # 1. Instanciar el cliente de Azure OpenAI
        # LangChain manejará internamente la validación de la conexión y el modelo al momento de usarlo.
        llm = AzureChatOpenAI(
            azure_deployment=deployment,
            azure_endpoint=endpoint,
            openai_api_key=api_key,
            api_version=api_version,
            temperature=temperature
        )

        # 2. Generar la respuesta (usando el método de streaming para ser similar al original)
        # El método stream devuelve un iterador de chunks de respuesta.
        output = ""
        # Creamos un mensaje con el formato que LangChain espera
        message = HumanMessage(content=prompt)
        
        for chunk in llm.stream([message]):
            # Cada 'chunk' es un objeto, extraemos su contenido de tipo string
            output += chunk.content
        
        return output.strip()

    except Exception as e:
        # La excepción puede ser por mala autenticación, endpoint incorrecto, despliegue no encontrado, etc.
        return f"ERROR: Ocurrió un error al contactar con Azure OpenAI. Detalles: {e}"

# --- USO DE LA FUNCIÓN ---

# Tus credenciales y configuración
endpoint = "https://aleja-mavdk5nu-eastus2.cognitiveservices.azure.com/"
deployment = "gpt-4o-mini"
# ATENCIÓN: Es una mala práctica tener claves secretas en el código.
# Considera usar variables de entorno.
subscription_key = 'D55vuDUYoqo7Bzj9R0SwUhQFA8MZ9l2Eh5yWCOfKSlMXLmKw7JBpJQQJ99BEACHYHv6XJ3w3AAAAACOGRJuO'
api_version = "2024-02-15-preview" # Usando una versión estable común para el chat

# Llamamos a la nueva función con la configuración
respuesta = generate_from_azure_openai(
    prompt="Explícame brevemente qué es Retrieval Augmented Generation",
    deployment=deployment,
    endpoint=endpoint,
    api_key=subscription_key,
    api_version=api_version
)

print(respuesta)

ERROR: Ocurrió un error al contactar con Azure OpenAI. Detalles: name 'AzureChatOpenAI' is not defined


In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
# docs = text_splitter.split_documents(data_pdf)

# for i in range(0, len(docs), 10):  # Procesa de 10 en 10
#     chunk = docs[i:i + 10]
#     vectorstore = Chroma.from_documents(
#         documents=chunk,
#         embedding=embedding,
#         persist_directory="./chroma_db_dir"
#     )
#     print(f"Procesados {len(chunk)} documentos y guardados en Chroma.")


Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en Chroma.
Procesados 10 documentos y guardados en 

In [23]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

chorma_db = Chroma(
    embedding_function=embedding,
    persist_directory="./chroma_db_dir"
)

retriever = chorma_db.as_retriever()
texto = """Tú eres un asistente para tareas de respuesta a preguntas."
    "Usa los siguientes fragmentos de contexto recuperado para responder "
    "la pregunta. Si no sabes la respuesta, di que no "
    "sabes. Usa un máximo de tres oraciones y mantén la "
    "respuesta concisa."""
    

def prompt(texto):
    system_prompt = (
    texto+
    "\n\n"
    "{context}"
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ])
    return prompt

def respuesta(pregunta, llm, chroma_db, prompt):
    retriever = chroma_db.as_retriever()

    chain = create_stuff_documents_chain(llm, prompt)
    rag = create_retrieval_chain(retriever, chain)
    
    results = rag.invoke({"input": pregunta})
    return results



In [24]:
pregunta = "¿Que sabes de la omega 3?"
resultado = respuesta(pregunta, llm, chorma_db, prompt(texto))
print(f"Resultado: {resultado['answer']}")

Resultado: El Omega 3 es un ácido graso poliinsaturado esencial que debe ser incorporado a través de la alimentación, y está formado por tres fracciones: ALA, EPA y DHA. Se recomienda su suplementación en personas que no consumen pescado, semillas o frutos secos de manera habitual, así como en aquellos con dietas muy bajas en calorías o con niveles elevados de triglicéridos y presión arterial. Las dosis recomendadas son de 1.6 g/día para hombres adultos y 1.1 g/día para mujeres adultas.


In [5]:
import chromadb

persistent_client = chromadb.PersistentClient(path='../chroma_db_dir')

In [4]:
from langchain.docstore.document import Document
from langchain_community.document_loaders import PyMuPDFLoader
from pathlib import Path
import re

def generate_from_azure_openai(prompt: str, llm_instance) -> str:
    """
    Genera una respuesta a partir de un prompt usando una instancia de AzureChatOpenAI.
    """
    try:
        # El método invoke es más simple para una única llamada no-streaming
        response = llm_instance.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        # En caso de error, devolvemos un string de error para poder manejarlo
        print(f"ERROR durante la generación: {e}")
        return f"ERROR_GENERACION: {e}"


def create_or_extract_abstracts(pdf_paths, llm_instance):
    """
    Extrae el resumen (abstract) de un PDF. Si no existe, lo genera usando un LLM.
    """
    abstract_docs = []
    
    # Un prompt de alta calidad para la tarea de resumir
    summarization_prompt_template = """
    Eres un asistente experto en la síntesis de documentos técnicos y académicos.
    A continuación, te proporciono el texto de un documento. Tu tarea es generar un
    resumen conciso (un 'abstract') en español, de no más de 250 palabras.

    El resumen debe capturar las ideas principales, la metodología (si la hay) y
    las conclusiones clave del texto. Los que hablen de suplementación añade en el resumen toda 
    la suplementación de la que se hable. Debe ser denso en información y estar
    escrito en un estilo formal y objetivo.

    Texto del documento:
    ---
    {document_text}
    ---
    """

    for path in pdf_paths:
        print(f"--- Procesando: {path} ---")
        try:
            loader = PyMuPDFLoader(str(path))
            pages = loader.load()
            full_text = " ".join([page.page_content for page in pages])
            
            # 1. INTENTAR EXTRAER EL ABSTRACT EXISTENTE
            start_match = re.search(r'(Abstract|Resumen)\b', full_text, re.IGNORECASE)
            abstract_text = ""

            if start_match:
                start_index = start_match.end()
                end_match = re.search(r'\b(Introduction|Introducción|Keywords|Palabras clave)\b', full_text[start_index:], re.IGNORECASE)
                if end_match:
                    end_index = start_index + end_match.start()
                    abstract_text = full_text[start_index:end_index].strip()
                else:
                    abstract_text = full_text[start_index:start_index+2500].strip()

            # 2. SI NO SE EXTRAJO, GENERARLO
            if not abstract_text or len(abstract_text) < 100: # Si es muy corto, probablemente no sea un buen abstract
                print(f"No se encontró abstract en '{path}'. Generando uno nuevo con Azure OpenAI...")

                # gpt-4o-mini tiene 128k tokens
                # 4 caracteres ~= 1 token. 100,000 caracteres ~= 25,000 tokens, más que seguro.
                max_chars = 100000
                truncated_text = full_text[:max_chars]

                prompt_para_resumir = summarization_prompt_template.format(document_text=truncated_text)
                
                generated_abstract = generate_from_azure_openai(prompt_para_resumir, llm_instance)
                
                if "ERROR_GENERACION" not in generated_abstract:
                    abstract_text = generated_abstract
                    print("Abstract generado con éxito.")
                else:
                    print(f"Fallo al generar abstract para {path}. Saltando documento.")
                    continue # Saltamos al siguiente documento
            else:
                print("Abstract extraído con éxito del documento.")

            # 3. CREAR EL DOCUMENTO DE LANGCHAIN
            doc = Document(
                page_content=abstract_text,
                metadata={
                    'title': path,
                    'source': str(path)
                }
            )
            print(doc.metadata)
            abstract_docs.append(doc)
        except Exception as e:
            print(f"Error fatal procesando {path}: {e}")
            
    return abstract_docs

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyMuPDFLoader
from pathlib import Path

def get_unique_sources_list(chroma_settings):
    # Obtén los datos de la colección
    collection_data = chroma_settings.get_collection('langchain').get(include=['embeddings', 'documents', 'metadatas'])
    
    # Extrae los metadatos
    metadatas = collection_data['metadatas']
    # Obtén los valores únicos de 'source'
    sources = set()
    for metadata in metadatas:
        source = metadata.get('source', None)
        if source:
            sources.add(source)
    
    # Obtener solo el nombre de archivo de cada ruta
    file_names = list(set(Path(source).name for source in sources))
    files = []
    for file_name in file_names:
        files.append("data/pdf/" + file_name)
        
    return files

def update_abstract_db(abstract_documents, embedding, abstract_db_dir="../chroma_db_abstracts_dir"):
    """
    Actualiza la base de datos de abstracts en Chroma.
    Si la base de datos ya existe, se actualiza con los nuevos documentos.
    """
    # Verifica si el directorio ya existe
    if Path(abstract_db_dir).exists():
        db_abstracts = Chroma(
            embedding_function=embedding,
            persist_directory=abstract_db_dir
        )
        db_abstracts.add_documents(abstract_documents)
    else:
        print(f"No existe la base de datos: '{abstract_db_dir}'")
        # Crea la base de datos
        db_abstracts = Chroma.from_documents(
            documents=abstract_documents,
            embedding=embedding,
            persist_directory=abstract_db_dir
        )
    
    return db_abstracts

def add_file_to_vectordb(filepath, embedding, llm, full_chroma_db_dir, abtracts_chorma_db_dir):
    abstrabs_doc = create_or_extract_abstracts([filepath], llm)
    update_abstract_db(abstrabs_doc, embedding, abtracts_chorma_db_dir)
    
    loader = PyMuPDFLoader(filepath)
    print(f"Cargando {filepath}...")
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
    splits = text_splitter.split_documents(docs)
    for i in range(0, len(docs), 10):  # Procesa de 10 en 10
        chunk = splits[i:i + 10]
        vectorstore = Chroma.from_documents(
            documents=chunk,
            embedding=embedding,
            persist_directory=full_chroma_db_dir
        )
    print(f"Procesados {len(chunk)} documentos y guardados en Chroma.") 



In [ ]:
def add_files_to_vectordb(persistent_client, embedding, llm, full_chroma_db_dir="../chroma_db_dir", abtracts_chorma_db_dir="../chroma_db_abstracts_dir"):
    data_folder = Path("data/pdf")

    pdf_paths = list(data_folder.glob("*.pdf"))

    cargados = get_unique_sources_list(persistent_client)

    nuevos = 0
    for path in pdf_paths:
        if path.as_posix() not in cargados:
            nuevos += 1
            add_file_to_vectordb(path.as_posix(), embedding, llm, full_chroma_db_dir, abtracts_chorma_db_dir)
    print(f"Se cargaron {nuevos} nuevos documentos de PDFs.")

In [ ]:
# add_files_to_vectordb(persistent_client, embedding, llm)
# get_unique_sources_list(persistent_client)
# Ver lista de colecciones




Colecciones disponibles: [Collection(name=langchain)]
Error accediendo a la colección: list indices must be integers or slices, not str


## Bases de datos

In [ ]:
# SQL (creación y consulta de base de datos relacional local)
!pip install sqlalchemy sqlite-utils

# Soporte para archivos Excel (.xlsx)
!pip install openpyxl pandas

In [ ]:
file_name = "food_cleaned.csv"
file_path = f"../data/csv/{file_name}"
import pandas as pd
import os
import sqlite3

if not os.path.exists(file_path):
    raise FileNotFoundError(f"Archivo no encontrado: {file_path}")

df = pd.read_csv(file_path)

# Conexión a una base de datos local SQLite
conn = sqlite3.connect("comida.db")
df.to_sql("comida", conn, if_exists="replace", index=False)

consulta_sql = pd.read_sql("SELECT * FROM comida LIMIT 5", conn)

## Creación RAG Abstracts

A continuación crearé un RAG que lo que realice sea extraer los abstracts de los documentos que ha recibido y se almacenen junto a su título, en caso de que no haya abstract o resumen en el documento se llamará a openAI para que genere el abstract a partir del documento completo.

In [ ]:
from langchain.docstore.document import Document
from langchain_community.document_loaders import PyMuPDFLoader
from pathlib import Path
import re

def generate_from_azure_openai(prompt: str, llm_instance) -> str:
    """
    Genera una respuesta a partir de un prompt usando una instancia de AzureChatOpenAI.
    """
    try:
        # El método invoke es más simple para una única llamada no-streaming
        response = llm_instance.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        # En caso de error, devolvemos un string de error para poder manejarlo
        print(f"ERROR durante la generación: {e}")
        return f"ERROR_GENERACION: {e}"


def create_or_extract_abstracts(pdf_paths, llm_instance):
    """
    Extrae el resumen (abstract) de un PDF. Si no existe, lo genera usando un LLM.
    """
    abstract_docs = []
    
    # Un prompt de alta calidad para la tarea de resumir
    summarization_prompt_template = """
    Eres un asistente experto en la síntesis de documentos técnicos y académicos.
    A continuación, te proporciono el texto de un documento. Tu tarea es generar un
    resumen conciso (un 'abstract') en español, de no más de 250 palabras.

    El resumen debe capturar las ideas principales, la metodología (si la hay) y
    las conclusiones clave del texto. Debe ser denso en información y estar
    escrito en un estilo formal y objetivo.

    Texto del documento:
    ---
    {document_text}
    ---
    """

    for path in pdf_paths:
        print(f"--- Procesando: {path.name} ---")
        try:
            loader = PyMuPDFLoader(str(path))
            pages = loader.load()
            full_text = " ".join([page.page_content for page in pages])
            
            # 1. INTENTAR EXTRAER EL ABSTRACT EXISTENTE
            start_match = re.search(r'(Abstract|Resumen)\b', full_text, re.IGNORECASE)
            abstract_text = ""

            if start_match:
                start_index = start_match.end()
                end_match = re.search(r'\b(Introduction|Introducción|Keywords|Palabras clave)\b', full_text[start_index:], re.IGNORECASE)
                if end_match:
                    end_index = start_index + end_match.start()
                    abstract_text = full_text[start_index:end_index].strip()
                else:
                    abstract_text = full_text[start_index:start_index+2500].strip()

            # 2. SI NO SE EXTRAJO, GENERARLO
            if not abstract_text or len(abstract_text) < 100: # Si es muy corto, probablemente no sea un buen abstract
                print(f"No se encontró abstract en '{path.name}'. Generando uno nuevo con Azure OpenAI...")

                # gpt-4o-mini tiene 128k tokens
                # 4 caracteres ~= 1 token. 100,000 caracteres ~= 25,000 tokens, más que seguro.
                max_chars = 100000
                truncated_text = full_text[:max_chars]

                prompt_para_resumir = summarization_prompt_template.format(document_text=truncated_text)
                
                generated_abstract = generate_from_azure_openai(prompt_para_resumir, llm_instance)
                
                if "ERROR_GENERACION" not in generated_abstract:
                    abstract_text = generated_abstract
                    print("Abstract generado con éxito.")
                else:
                    print(f"Fallo al generar abstract para {path.name}. Saltando documento.")
                    continue # Saltamos al siguiente documento
            else:
                print("Abstract extraído con éxito del documento.")

            # 3. CREAR EL DOCUMENTO DE LANGCHAIN
            doc = Document(
                page_content=abstract_text,
                metadata={
                    'title': path.name,
                    'source': str(path)
                }
            )
            abstract_docs.append(doc)

        except Exception as e:
            print(f"Error fatal procesando {path.name}: {e}")
            
    return abstract_docs

In [ ]:
# 1. Instanciar modelo LLM de Azure OpenAI
llm = AzureChatOpenAI(
    openai_api_key=subscription_key,
    azure_endpoint=endpoint,
    api_version=api_version,
    azure_deployment=deployment,
    temperature=0
)

# 2. Definir la lista PDFs a procesar
data_folder = Path("../data/pdf")
pdf_paths = list(data_folder.glob("*.pdf"))

# 3. Llama a la función que extrae O crea los abstracts
abstract_documents = create_or_extract_abstracts(pdf_paths, llm)

print(f"\nProceso completado. Se obtuvieron {len(abstract_documents)} abstracts (extraídos o generados).")


--- Procesando: actualizacionDeporteAdolescentesYNiños.pdf ---
Abstract extraído con éxito del documento.
--- Procesando: Alimentacion-nutricion-e-hidratacion-en-el-deporte.pdf ---
No se encontró abstract en 'Alimentacion-nutricion-e-hidratacion-en-el-deporte.pdf'. Generando uno nuevo con Azure OpenAI...
Abstract generado con éxito.
--- Procesando: Assessing gender differences in food preferences and physical activity.pdf ---
No se encontró abstract en 'Assessing gender differences in food preferences and physical activity.pdf'. Generando uno nuevo con Azure OpenAI...
Abstract generado con éxito.
--- Procesando: devries-jakobi-2021-importance-of-considering-sex-and-gender-in-exercise-and-nutrition-research.pdf ---
No se encontró abstract en 'devries-jakobi-2021-importance-of-considering-sex-and-gender-in-exercise-and-nutrition-research.pdf'. Generando uno nuevo con Azure OpenAI...
Abstract generado con éxito.
--- Procesando: Dialnet-EfectosDelEntrenamientoDeFuerzaSobreLaCondicionFis-80

In [ ]:
# Directorio para la BD de abstracts
abstract_db_dir = "../chroma_db_abstracts_dir"



update_abstract_db(abstract_documents, embedding, abstract_db_dir)

print(f"Creando base de datos vectorial para los abstracts en '{abstract_db_dir}'...")
db_abstracts = Chroma.from_documents(
    documents=abstract_documents,
    embedding=embedding, # Tu 'embedding' de Azure
    persist_directory=abstract_db_dir
)
print("Base de datos de abstracts creada/actualizada.")

Creando base de datos vectorial para los abstracts en './chroma_db_abstracts_dir'...
Base de datos de abstracts creada/actualizada.


In [6]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

def resolver_rag(pregunta, llm, prompt_template, db_full_path, db_abstracts_path):
    """
    Responde una pregunta utilizando un RAG de dos pasos.
    """
    # --- PASO 1: Buscar en la base de datos de abstracts ---
    print("Paso 1: Buscando en la base de datos de abstracts...")
    db_abstracts = Chroma(persist_directory=db_abstracts_path, embedding_function=embedding)
    retriever_abstracts = db_abstracts.as_retriever(search_kwargs={"k": 5}) # Buscamos los 3 abstracts más relevantes
    
    relevant_abstracts = retriever_abstracts.invoke(pregunta)
    
    # Extraemos los títulos (nombres de archivo) de los documentos relevantes
    relevant_sources = list(set([doc.metadata['source'] for doc in relevant_abstracts]))
    
    if not relevant_sources:
        return {"answer": "No pude encontrar documentos relevantes en los resúmenes para responder la pregunta."}
        
    print(f"Documentos relevantes encontrados por título: {relevant_sources}")

    # --- PASO 2: Buscar en la base de datos principal usando los títulos como filtro ---
    print("Paso 2: Buscando en la base de datos principal con filtro de rutas...")
    db_full = Chroma(persist_directory=db_full_path, embedding_function=embedding)
    
    # Creamos un retriever filtrado
    # Solo buscará en los documentos cuyo metadato 'source' contenga alguno de los nombres de archivo
    retriever_full_filtered = db_full.as_retriever(
        search_kwargs={
            "k": 8, # Tomamos 5 chunks de los documentos filtrados
            "filter": {
                "source": {"$in": relevant_sources}
            }
        }
    )

    # Creamos la cadena de RAG final con el retriever filtrado
    create_chain = create_stuff_documents_chain(llm, prompt_template)
    rag_chain_filtered = create_retrieval_chain(retriever_full_filtered, create_chain)
    
    # Invocamos la cadena para obtener la respuesta final
    filtered_result = rag_chain_filtered.invoke({"input": pregunta})
    
     # --- INICIO DE LA NUEVA LÓGICA DE FALLBACK ---
     # Palabras clave para detectar si el LLM no encontró la respuesta en el contexto filtrado.
    negative_keywords = [
        "no tengo la respuesta", 
        "no se menciona", 
        "no encuentro información",
        "información no está presente",
        "contexto proporcionado no",
        "No sé",
        "no sé la respuesta",
        "no sé"
    ]
    
    
    # Comprobamos si alguna de las palabras clave está en la respuesta (en minúsculas)
    is_negative_answer = any(keyword in filtered_result['answer'].lower() for keyword in negative_keywords)
    
    if is_negative_answer:
        print("\nPASO 3: La respuesta inicial fue insuficiente. Realizando búsqueda de respaldo en TODA la base de datos...")
        
        # Creamos un nuevo retriever SIN FILTRO para buscar en todos los documentos
        retriever_full = db_full.as_retriever(search_kwargs={"k": 10})
        
        # Creamos una nueva cadena RAG con el retriever sin filtro
        rag_chain = create_retrieval_chain(retriever_full, create_chain)
        
        resultado_fallback = rag_chain.invoke({"input": pregunta})
        
        return resultado_fallback
    else:
        return filtered_result

# --- Uso de la función ---

# Definimos el texto del prompt (como ya lo tenías)
texto_prompt = """Tú eres un asistente experto en nutrición deportiva, suplementación y entrenamiento
físico, diseñado para responder preguntas con la máxima precisión. Utiliza los siguientes fragmentos 
de contexto recuperado para responder a la pregunta del usuario. Si la información no está presente 
en el contexto proporcionado, indica claramente que no tienes la respuesta, respondiendo No sé. Limita tu respuesta a un 
máximo de cinco oraciones, asegurando que sea concisa y directamente relevante para el ámbito del 
deporte y la salud.

{context}
"""
prompt_template = ChatPromptTemplate.from_messages([
    ("system", texto_prompt),
    ("human", "{input}"),
])

# Tu pregunta
pregunta = "¿Cuál es el mejor suplemento si estás bajo en energía y quieres aumentar tu rendimiento en algún deporte?"

# Ejecutamos el RAG de dos pasos
resultado_final = resolver_rag(
    pregunta=pregunta,
    llm=llm,
    prompt_template=prompt_template,
    db_full_path="../chroma_db_dir",
    db_abstracts_path="../chroma_db_abstracts_dir"
)

print("\n--- Respuesta Final ---")
print(resultado_final['answer'])

Paso 1: Buscando en la base de datos de abstracts...


C:\Users\asusg\AppData\Local\Temp\ipykernel_6156\566184713.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db_abstracts = Chroma(persist_directory=db_abstracts_path, embedding_function=embedding)


Documentos relevantes encontrados por título: ['data/pdf/efectosEntrenamientoFuerzaEnDeporteEquipo.pdf', 'data/pdf/Comparative-effects-of-caffeine-beta‑alanine.pdf', 'data/pdf/nutricionYSaludDietaVegana.pdf', 'data/pdf/estrategiasDosisMinimas.pdf', 'data/pdf/Dialnet-EfectosDelEntrenamientoDeFuerzaSobreLaCondicionFis-8053359.pdf']
Paso 2: Buscando en la base de datos principal con filtro de rutas...

PASO 3: La respuesta inicial fue insuficiente. Realizando búsqueda de respaldo en TODA la base de datos...

PASO 3: La respuesta inicial fue insuficiente. Realizando búsqueda de respaldo en TODA la base de datos...

--- Respuesta Final ---
No sé.

--- Respuesta Final ---
No sé.


In [8]:

pregunta = "¿Qué sabes sobre el omega 3 y su suplementación?"
resultado_final = resolver_rag(
    pregunta=pregunta,
    llm=llm,
    prompt_template=prompt_template,
    db_full_path="../chroma_db_dir",
    db_abstracts_path="../chroma_db_abstracts_dir")
print(resultado_final['answer'])

Paso 1: Buscando en la base de datos de abstracts...
Documentos relevantes encontrados por título: ['data/pdf/Regulation_of_Muscle_Glycogen_Metabolism_during_Ex.pdf', 'data/pdf/efectosSuplementacionNitratosRendimientoDeportivo.pdf', 'data/pdf/AlimentacionVeganaRendimientoDeportivo.pdf', 'data/pdf/Monografico-Ashwagandha.pdf', 'data/pdf/estrategiasDosisMinimas.pdf']
Paso 2: Buscando en la base de datos principal con filtro de rutas...

PASO 3: La respuesta inicial fue insuficiente. Realizando búsqueda de respaldo en TODA la base de datos...
El Omega 3 es un ácido graso poliinsaturado esencial que debe ser incorporado a través de la alimentación, y se encuentra en pescados azules, semillas, aceite de lino y frutos secos. Se recomienda la suplementación en personas que no consumen pescado, en dietas muy bajas en calorías, o cuando los niveles de triglicéridos y la presión arterial son elevados. La dosis recomendada es de 1.6 g/día para hombres adultos y 1.1 g/día para mujeres adultas. Par

In [2]:
import time

In [10]:
preguntas_basicas = [
    "¿Qué es la creatina y para qué sirve en el deporte?",
    "¿Cuál es la función de la proteína whey?",
    "¿Qué diferencia hay entre BCAA y EAA?",
    "¿Cuándo es mejor tomar proteína: antes o después de entrenar?",
    "¿Es necesario tomar suplementos si ya tengo una dieta equilibrada?"
]

for pregunta in preguntas_basicas:
    print(f"\nPregunta: {pregunta}")
    resultado_final = resolver_rag(
        pregunta=pregunta,
        llm=llm,
        prompt_template=prompt_template,
        db_full_path="../chroma_db_dir",
        db_abstracts_path="../chroma_db_abstracts_dir"
    )
    print(f"Respuesta: {resultado_final['answer']}")
    print("\n")
    time.sleep(1)


Pregunta: ¿Qué es la creatina y para qué sirve en el deporte?
Paso 1: Buscando en la base de datos de abstracts...
Documentos relevantes encontrados por título: ['data/pdf/Regulation_of_Muscle_Glycogen_Metabolism_during_Ex.pdf', 'data/pdf/ejercicio-fisico-y-nutricion-Marcela-Gonzalez-Gross.pdf', 'data/pdf/AlimentacionVeganaRendimientoDeportivo.pdf', 'data/pdf/Monografico-Ashwagandha.pdf', 'data/pdf/entrenamientoDeFuerzaYLaIncidenciaEjercicios.pdf']
Paso 2: Buscando en la base de datos principal con filtro de rutas...
Respuesta: La creatina es un compuesto que se encuentra en el tejido muscular y se produce naturalmente en el cuerpo a partir de aminoácidos. En el deporte, la creatina se utiliza como un suplemento para mejorar el rendimiento, especialmente en actividades de alta intensidad y corta duración, como el levantamiento de pesas y los sprints. Su función principal es aumentar las reservas de fosfocreatina en los músculos, lo que ayuda a regenerar ATP, la principal fuente de ene

In [ ]:
preguntas_intermedias = [
    "¿Cuáles son los beneficios del omega-3 en el rendimiento deportivo?",
    "¿La cafeína realmente mejora el rendimiento en entrenamientos de fuerza?",
    "¿Qué tipo de proteína es mejor para ganar masa muscular?",
    "¿Cuál es la dosis recomendada de beta-alanina para deportistas?",
    "¿Qué efectos secundarios puede tener el consumo prolongado de pre-entrenos?"
]

for pregunta in preguntas_intermedias:
    print(f"\nPregunta: {pregunta}")
    resultado_final = resolver_rag(
        pregunta=pregunta,
        llm=llm,
        prompt_template=prompt_template,
        db_full_path="../chroma_db_dir",
        db_abstracts_path="../chroma_db_abstracts_dir"
    )
    print(f"Respuesta: {resultado_final['answer']}")
    print("\n")
    time.sleep(1)


Pregunta: ¿Cuáles son los beneficios del omega-3 en el rendimiento deportivo?
Paso 1: Buscando en la base de datos de abstracts...
Documentos relevantes encontrados por título: ['data\\pdf\\entrenamientoDeFuerzaYLaIncidenciaEjercicios.pdf', 'data\\pdf\\Alimentacion-nutricion-e-hidratacion-en-el-deporte.pdf', 'data\\pdf\\EfectosDeLaSuplementacionConCreatinaMonohidratada.pdf', 'data\\pdf\\efectosEntrenamientoJovenes.pdf', 'data\\pdf\\Regulation_of_Muscle_Glycogen_Metabolism_during_Ex.pdf', 'data\\pdf\\nutricionEntrenamientoYCompeticion.pdf', 'data\\pdf\\NutricionSuplementacionEHidratacionEnElAmbitoDep.pdf', 'data/pdf/efectosCafeinaRendimientoDeportivo.pdf']
Paso 2: Buscando en la base de datos principal con filtro de rutas...

PASO 3: La respuesta inicial fue insuficiente. Realizando búsqueda de respaldo en TODA la base de datos...
Respuesta: El omega-3 es un ácido graso esencial que ofrece varios beneficios en el rendimiento deportivo, incluyendo su efecto antiinflamatorio y antioxidan

In [ ]:
preguntas_avanzadas = [
    "¿Cómo influye la suplementación con HMB en la prevención del catabolismo muscular?",
    "¿Qué evidencia científica respalda el uso de ashwagandha en la reducción del cortisol en atletas?",
    "¿En qué casos no se recomienda el uso de suplementos de nitratos como la remolacha?",
    "¿Qué interacción hay entre la vitamina D y el metabolismo del calcio en atletas?",
    "¿Cómo influye la crononutrición en la eficacia de los suplementos deportivos?",
    "¿Qué dieta optimiza la diabetes tipo 2?"
]

for pregunta in preguntas_avanzadas:
    print(f"\nPregunta: {pregunta}")
    resultado_final = resolver_rag(
        pregunta=pregunta,
        llm=llm,
        prompt_template=prompt_template,
        db_full_path="../chroma_db_dir",
        db_abstracts_path="../chroma_db_abstracts_dir"
    )
    print(f"Respuesta: {resultado_final['answer']}")
    print("\n")
    time.sleep(1)


Pregunta: ¿Cómo influye la suplementación con HMB en la prevención del catabolismo muscular?
Paso 1: Buscando en la base de datos de abstracts...
Documentos relevantes encontrados por título: ['data\\pdf\\entrenamientoDeFuerzaYLaIncidenciaEjercicios.pdf', 'data\\pdf\\entrenamientoAlFallo.pdf', 'data\\pdf\\EfectosDeLaSuplementacionConCreatinaMonohidratada.pdf', 'data\\pdf\\Regulation_of_Muscle_Glycogen_Metabolism_during_Ex.pdf', 'data\\pdf\\nutricionEntrenamientoYCompeticion.pdf']
Paso 2: Buscando en la base de datos principal con filtro de rutas...
Respuesta: La suplementación con β-hidroxi-β-metilbutirato (HMB) se considera un agente anti-catabólico que puede ayudar a aumentar la ganancia de fuerza y masa magra asociada al entrenamiento de resistencia. Sin embargo, los resultados de los estudios son contradictorios, mostrando un aumento leve en la masa magra y fuerza, principalmente en deportistas principiantes durante las primeras 2-4 semanas de entrenamiento. A medida que los indiv

In [ ]:
preguntas_contexto = [
    "Soy un corredor de maratón, ¿qué suplemento me conviene tomar durante la fase de carga?",
    "¿Qué tipo de suplemento es mejor para una mujer de 50 años que entrena fuerza y busca mantener masa muscular?",
    "Entreno en ayunas por la mañana, ¿debo tomar BCAA antes o después?",
    "Llevo una dieta vegana, ¿qué suplementos deportivos necesito considerar?",
    "He empezado a tomar creatina, ¿debo hacer fase de carga o no es necesario?"
]

for pregunta in preguntas_contexto:
    print(f"\nPregunta: {pregunta}")
    resultado_final = resolver_rag(
        pregunta=pregunta,
        llm=llm,
        prompt_template=prompt_template,
        db_full_path="../chroma_db_dir",
        db_abstracts_path="../chroma_db_abstracts_dir"
    )
    print(f"Respuesta: {resultado_final['answer']}")
    print("\n")
    time.sleep(1)


Pregunta: Soy un corredor de maratón, ¿qué suplemento me conviene tomar durante la fase de carga?
Paso 1: Buscando en la base de datos de abstracts...
Documentos relevantes encontrados por título: ['data\\pdf\\Alimentacion-nutricion-e-hidratacion-en-el-deporte.pdf', 'data\\pdf\\Regulation_of_Muscle_Glycogen_Metabolism_during_Ex.pdf', 'data\\pdf\\nutricionEntrenamientoYCompeticion.pdf', 'data\\pdf\\NutricionSuplementacionEHidratacionEnElAmbitoDep.pdf', 'data\\pdf\\La-guía-completa-de-la-nutrición-del-deportista.pdf']
Paso 2: Buscando en la base de datos principal con filtro de rutas...
Respuesta: Durante la fase de carga para un maratón, es recomendable enfocarse en la carga de hidratos de carbono para maximizar los depósitos de glucógeno muscular. Además, la cafeína puede ser beneficiosa, ya que puede mejorar el rendimiento y la percepción del esfuerzo. La creatina también puede ser considerada, aunque su uso es más común en disciplinas de corta duración y alta intensidad. Sin embargo

In [7]:
preguntas_nuevas = [
    "¿Qué suplementos son ideales para mejorar la recuperación muscular después de entrenamientos intensos?",
    "¿Cómo afecta la suplementación con magnesio al rendimiento deportivo?",
    "¿Qué evidencia científica respalda el uso de antioxidantes en atletas de resistencia?",
    "¿Cuál es la mejor estrategia de hidratación para corredores de larga distancia?",
    "¿Qué suplementos son recomendados para deportistas veganos que buscan aumentar masa muscular?"
 ]

for pregunta in preguntas_nuevas:
    print(f"\nPregunta: {pregunta}")
    resultado_final = resolver_rag(
        pregunta=pregunta,
        llm=llm,
        prompt_template=prompt_template,
        db_full_path="../chroma_db_dir",
        db_abstracts_path="../chroma_db_abstracts_dir"
    )
    print(f"Respuesta: {resultado_final['answer']}")
    print("\n")
    time.sleep(1)


Pregunta: ¿Qué suplementos son ideales para mejorar la recuperación muscular después de entrenamientos intensos?
Paso 1: Buscando en la base de datos de abstracts...
Documentos relevantes encontrados por título: ['data/pdf/Monografico-Ashwagandha.pdf', 'data/pdf/Comparative-effects-of-caffeine-beta‑alanine.pdf', 'data/pdf/gananciaFuerzaConDiferenciaEntreSeries.pdf', 'data/pdf/effectCaffeineOnAtheticPerformance.pdf', 'data/pdf/entrenamientoDeFuerzaYLaIncidenciaEjercicios.pdf']
Paso 2: Buscando en la base de datos principal con filtro de rutas...
Documentos relevantes encontrados por título: ['data/pdf/Monografico-Ashwagandha.pdf', 'data/pdf/Comparative-effects-of-caffeine-beta‑alanine.pdf', 'data/pdf/gananciaFuerzaConDiferenciaEntreSeries.pdf', 'data/pdf/effectCaffeineOnAtheticPerformance.pdf', 'data/pdf/entrenamientoDeFuerzaYLaIncidenciaEjercicios.pdf']
Paso 2: Buscando en la base de datos principal con filtro de rutas...

PASO 3: La respuesta inicial fue insuficiente. Realizando búsq